In [1]:
import openai


In [ ]:
import openai_secret_manager
import openai
from bs4 import BeautifulSoup
import requests
import json

# Fetch webpage text
prof = ["jeffe", "vadve", "agewirth"]
url = f"https://cs.illinois.edu/about/people/all-faculty/{prof[0]}"
response = requests.get(url)
html_text = response.text

# Initialize OpenAI API client
secrets = openai_secret_manager.get_secret("openai")
openai.api_key = secrets["api_key"]

# Define prompt to extract information
prompt = (f"Extract the professor's name, education, and research interests from the following webpage:\n\n{html_text}\n\n"
          "Name:\nEducation:\nInterests:")

# Call OpenAI's GPT-3 to extract information
response = openai.Completion.create(
    engine="text-davinci-002",
    prompt=prompt,
    max_tokens=3072,
    n=1,
    stop=None,
    temperature=0.5,
)

# Extract the generated text from the response
result = response.choices[0].text.strip()

# Parse the result into a JSON object
data = {}
for line in result.split("\n"):
    if line.startswith("Name:"):
        data["name"] = line[len("Name:"):].strip()
    elif line.startswith("Education:"):
        data["education"] = line[len("Education:"):].strip()
    elif line.startswith("Interests:"):
        data["interests"] = line[len("Interests:"):].strip()

# Store the data in a JSON file
with open("professor_info.json", "w") as f:
    json.dump(data, f, indent=4)


In [16]:
from bs4 import BeautifulSoup
import requests
import re
prof = ["jeffe", "vadve", "agewirth"]
url = f"https://cs.illinois.edu/about/people/all-faculty/{prof[0]}"
response = requests.get(url)

soup = BeautifulSoup(response.content, 'html.parser')
html_text = soup.get_text(strip=True,separator= "\n")
#html_text = re.sub(r'\n\s*\n', '\n\n', html_text)

# # First, remove any new line characters
# html_text = html_text.replace("\n", "")

# # Then, replace multiple white spaces with a single space
# html_text = " ".join(html_text.split())

# # Finally, add line breaks in appropriate places
# html_text = html_text.replace("<", "\n<")
# html_text = html_text.replace(">", ">\n")

# Open a new text file for writing
with open("output.txt", "w") as file:
    # Write the formatted HTML text to the file
    file.write(html_text)

print(html_text)

Jeff Erickson
 | Computer Science | UIUC
University of Illinois Urbana-Champaign
Alumni
Corporate
People
My.CS
University of Illinois at Urbana-Champaign
The Grainger College of Engineering
Computer Science
Search
Menu
Search
About
About
Rankings & Statistics
Contact Us & Office Locations
History Timeline
Accreditation
Values & Code of Conduct
CS CARES Committee
Contact CS CARES
Governance
Members
Resources
People
All Faculty
Department Faculty
Affiliate Faculty
Adjunct Faculty
Emeritus Faculty
Postdoctoral Researchers
Staff
Office of the Department Head
Communications & Engagement Team
Undergraduate Advising Office
Graduate Advising Office
Instructional Development Team
Business Office
Faculty Support Contacts
Facilities, Shipping and Receiving
Graduating PhD Students
Open Positions
Faculty Positions
Postdoctoral Positions
Future Faculty Fellows
Staff Positions
Choose Illinois Computer Science
Awards
2021 Celebration of Excellence
2022 Celebration of Excellence
2023 Celebration of Exc

In [26]:
import requests
from bs4 import BeautifulSoup
import re
import nltk
from sumy.parsers.html import HtmlParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.utils import get_stop_words

# Initialize NLTK
#nltk.download('punkt')

# Set up Sumy summarizer
LANGUAGE = "english"
SENTENCES_COUNT = 27

# Set up web scraping
prof = ["jeffe", "vadve", "agewirth"]
url = f"https://cs.illinois.edu/about/people/all-faculty/{prof[0]}"
response = requests.get(url)

# Parse HTML with BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')
html_text = soup.get_text()

# Preprocess text with NLTK
sentences = nltk.sent_tokenize(html_text)
words = [nltk.word_tokenize(sentence) for sentence in sentences]

# Initialize Sumy summarizer
parser = HtmlParser.from_url(url, Tokenizer(LANGUAGE))
summarizer = LsaSummarizer()
summarizer.stop_words = get_stop_words(LANGUAGE)

# Summarize text with Sumy
summary = summarizer(parser.document, SENTENCES_COUNT)

# Print the summary
print("Summary:")
for sentence in summary:
    print(sentence)

# Open a new text file for writing
with open("output.txt", "w") as file:
    # Write the summarized text to the file
    file.write("Summary:\n")
    for sentence in summary:
        file.write(str(sentence) + "\n")


Summary:
I have published over 100 technical papers in computational geometry, computational topology, graph algorithms, and related topics at the intersection of computer science and mathematics.
Among many other conference committee memberships, I was the chair of the community-elected steering committee for the International Symposium on Computational Geometry (SOCG) from 2013 to 2016, and I am currently a SafeTOC(anti-harassment) advocate for SOCG and for the ACM-SIAM Symposium on Discrete Algorithms (SODA).
I primarily teach large algorithms classes; my free Algorithms textbook is used in dozens of computer science departments and by myriads of of students, recruiters, and professionals around the world.
At various times in my professional life I have been an associate department head, the chair of the faculty advisory committee, the manager of a software development project, an Apple II assembly-language programmer, a disk jockey, a pizza cook, and a knot-tying instructor at a Bo

In [30]:
import requests
import re
import nltk
from bs4 import BeautifulSoup
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer

# Download the necessary nltk resources
#nltk.download("punkt")

# Set up the URL and professor names
prof = ["jeffe", "vadve", "agewirth"]
url = f"https://cs.illinois.edu/about/people/all-faculty/{prof[0]}"

# Get the HTML content
response = requests.get(url)

# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

# Get the raw text from the HTML content
html_text = soup.get_text(strip=True, separator="\n")

# Use sumy to extract the most important sentences
parser = PlaintextParser.from_string(html_text, Tokenizer("english"))
summarizer = TextRankSummarizer()
summary = summarizer(parser.document, sentences_count=5)
important_sentences = [str(sentence) for sentence in summary]

# Prompt ChatGPT with the important sentences
prompt = f"Extract the professor's name, education, and research interests from the following webpage into json format:\n\n{' '.join(important_sentences)}\n\nName:\nEducation:\nInterests:"

# Print the prompt
print(prompt)
# Trifekta github


Extract the professor's name, education, and research interests from the following webpage into json format:

Jeff Erickson | Computer Science | UIUC University of Illinois Urbana-Champaign Alumni Corporate People My.CS University of Illinois at Urbana-Champaign The Grainger College of Engineering Computer Science Search Menu Search About About Rankings & Statistics Contact Us & Office Locations History Timeline Accreditation Values & Code of Conduct CS CARES Committee Contact CS CARES Governance Members Resources People All Faculty Department Faculty Affiliate Faculty Adjunct Faculty Emeritus Faculty Postdoctoral Researchers Staff Office of the Department Head Communications & Engagement Team Undergraduate Advising Office Graduate Advising Office Instructional Development Team Business Office Faculty Support Contacts Facilities, Shipping and Receiving Graduating PhD Students Open Positions Faculty Positions Postdoctoral Positions Future Faculty Fellows Staff Positions Choose Illinois 

In [31]:
from bs4 import BeautifulSoup
import requests
import re
prof = ["jeffe", "vadve", "agewirth"]
url = f"https://cs.illinois.edu/about/people/all-faculty/{prof[0]}"
response = requests.get(url)

html_text = response.text

# Open a new text file for writing
with open("output.txt", "w") as file:
    # Write the formatted HTML text to the file
    file.write(html_text)

print(html_text)

<!DOCTYPE html>
<html lang="en">
	<head>
		<meta charset="UTF-8">
		<title>Jeff Erickson
 | Computer Science | UIUC</title>
		<link rel="shortcut icon" type="image/x-icon" href="https://ws.engr.illinois.edu/images/favicon.ico">
		<link rel="canonical" href="https://cs.illinois.edu/about/people/all-faculty/jeffe" />

		<meta name="description" content="Jeff Erickson
">
		<meta name="author" content="Grainger Engineering Office of Marketing and Communications">

		<!-- meta tags for mobile devices -->
		<meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">
		<meta name="apple-mobile-web-app-capable" content="yes">  
		<meta name="apple-mobile-web-app-status-bar-style" content="black">
		<meta http-equiv="X-UA-Compatible" content="IE=edge">

				<script src="https://onetrust.techservices.illinois.edu/scripttemplates/otSDKStub.js" type="text/javascript" charset="UTF-8" data-domain-script="26be7d61-2017-4ea7-8a8b-8f1704889763"></script>
		<script type="text/